# Data Preprocessing: 2018-2020

Shayna Howitt

May 20, 2020

## Background

The below data was pulled by Grassroots Economics from their community currency program. The raw data can be found [here](https://www.grassrootseconomics.org/research). More information about how the datasets were created can be found [here](https://github.com/GrassrootsEconomics/CIC-Docs/blob/master/TransactionDatasets.md).

The first dataset is from 9/26/2018 to 1/1/2020. The second dataset is from 1/1/2020 to 5/11/2020. 

## Purpose

This notebook will determine how to most effectively combine the two (considering some data storage has changed), and then export a complete  file of all transactions. 

## Conclusions  
 
**New Data**
- In 2020, GE began storing the token name and transfer subtype for the first time. This gives us transparency into the purpose of the transfer, which we did not have before.

**Sources**
- It does not appear that any of the sources from 2018-2019 appear in the 2020 documents. Unknown why this is; requires future research. 

**Free Text**
- It appears there were more free text options released in 2020, as there were new variations of how to spell and write gender and locations. Many more locations appeared. There were just a few changes to business types. 

In [1]:
import numpy as np
import pandas as pd

In [2]:
transactions_1819 = pd.read_csv('POA_tx_all_pub.csv')
transactions_20 = pd.read_csv('2020_transactions.csv')

# 1. Data Formatting

## 1a. Column Names

**Did data names change?**

Yes, the name of the location columns changed. 

New columns: 
- token address: blockchain address
- token name: Sarafu or Kenyan Shilling
- transfer subtype: 
   - DISBURSMENT = from Grassroots Economics
   - RECLEMATION = Back to GE
   - STANDARD = a trade between users
   - AGENT = when a group account is cashing out

In [3]:
set(transactions_20.columns) ^ set(transactions_1819.columns)

{'s_comm_tkn',
 's_location',
 't_comm_tkn',
 't_location',
 'token_address',
 'token_name',
 'transfer_subtype',
 'tx_hash'}

In [4]:
new_location_cols = {
    's_comm_tkn': 's_location', 
    't_comm_tkn': 't_location'
}
transactions_1819.rename(columns=new_location_cols, inplace=True)

## 1b. Timeset

In [5]:
def transform_dates(date_str):
    _datetime_temp = pd.to_datetime(date_str)
    return _datetime_temp.date()

In [6]:
transactions_1819 = transactions_1819.assign(
    date=transactions_1819['timeset'].apply(transform_dates)
)

transactions_20 = transactions_20.assign(
    date=transactions_20['timeset'].apply(transform_dates)
)

## 1c. Sources

In [7]:
print('18-19:', len(transactions_1819['source'].unique()))
print('20:', len(transactions_20['source'].unique()))

18-19: 8325
20: 18493


In [8]:
shared_source = set(transactions_1819['source']) & (set(transactions_20['source']))
print(f'source in both: {len(shared_source)}')

source in both: 0


*Both sources have 42 characters.*

In [9]:
display(transactions_20['source'].apply(len).value_counts())
display(transactions_1819['source'].apply(len).value_counts())

42    147815
Name: source, dtype: int64

42    103691
Name: source, dtype: int64

*Doesn't share any sources even with all capitals*

In [10]:
shared_uppercase_source = set(transactions_1819['source'].apply(
    lambda x: x.upper())) & set(transactions_20['source'].apply(lambda x: x.upper()))
len(shared_uppercase_source)

0

*Are there no sources that carry over? They potentially changed how source ids are stored in 2020*

## 1d. Gender

In [11]:
display(transactions_20['s_gender'].unique())

array([nan, 'male', 'female', 'Male', 'Female', 'other', 'Unknown gender',
       'Other', 'Male '], dtype=object)

In [12]:
display(transactions_1819['s_gender'].unique())

array(['Male', nan, 'Female', 'Other'], dtype=object)

In [13]:
gender_col_mapping = {
    'female': 'Female',
    'male': 'Male', 
    'Male ': 'Male', 
    'other': 'Other', 
    'Unknown gender': np.NaN
}

In [14]:
transactions_20['s_gender'].replace(gender_col_mapping, inplace=True)
transactions_20['t_gender'].replace(gender_col_mapping, inplace=True)

## 1e. Location

In [15]:
locations = set(transactions_1819['t_location'].dropna().unique())

In [16]:
transactions_20['t_location'] = transactions_20['t_location'].fillna('NONE')
transactions_20['S_location'] = transactions_20['t_location'].fillna('NONE')

In [17]:
def return_formatted_location_match(input_location_str):
    if input_location_str == 'NONE':
        return np.NaN
    elif input_location_str in locations:
        return input_location_str 
    else:
        for loc in locations:
            if loc in input_location_str:
                return loc
        return input_location_str
            

In [18]:
transactions_20['t_location'] = transactions_20['t_location'].str.upper().str.strip().apply(return_formatted_location_match)
transactions_20['s_location'] = transactions_20['s_location'].str.upper().str.strip().apply(return_formatted_location_match)

## 1f. Business Types

In [19]:
business_type_col_mapping = {
    'Food': 'Food/Water', 
    'Labour': 'Farming/Labour', 
    np.NaN: 'None',
    'General shop': 'Shop', 
    'Energy': 'Fuel/Energy', 
    'Water': 'Food/Water'
}

In [20]:
transactions_1819['s_business_type'].replace(business_type_col_mapping, inplace=True)
transactions_1819['t_business_type'].replace(business_type_col_mapping, inplace=True)

# 2. Combine and save all data

In [21]:
use_cols = ['id', 'date', 'source', 's_location', 's_gender', 's_business_type',
       'target', 't_location', 't_gender', 't_business_type', 'tx_token',
       'weight', 'tx_hash', 'type']

In [22]:
all_transactions = pd.concat([transactions_1819, transactions_20])
all_transactions = all_transactions[use_cols]
all_transactions.head()

,id,date,source,s_location,s_gender,s_business_type,target,t_location,t_gender,t_business_type,tx_token,weight,tx_hash,type
0,1,2018-09-26,0x3cc06c32c88aba379c4bbd4c9ebda585d7574bbf,LINDI,Male,Other,0xa54b77e28e8954c8c89d234c262a71125cb98da8,NaN,NaN,None,GATINA,100.0,0x880d3b999efe44299f96678ec2a3758c168ed4a61b6b...,directed
1,2,2018-09-26,0x23c848671adf998025cd1d2247fea93f922b6d31,CONGO,Male,Food/Water,0x3cc06c32c88aba379c4bbd4c9ebda585d7574bbf,LINDI,Male,Other,GATINA,2.0,0x4893bc5024d1ef291f95318c86600e26b9054a273f5e...,directed
2,3,2018-09-26,0x3cc06c32c88aba379c4bbd4c9ebda585d7574bbf,LINDI,Male,Other,0x23c848671adf998025cd1d2247fea93f922b6d31,CONGO,Male,Food/Water,GATINA,20.0,0xf901ff660b432c7d3049edbfd07a26ace9264eae07e3...,directed
3,4,2018-09-26,0x3cc06c32c88aba379c4bbd4c9ebda585d7574bbf,LINDI,Male,Other,0xb7b1a85fed492df4a67f34a723408fc5f694f96b,NaN,NaN,None,GEP,20.0,0x2fc386334d728fadd0d62522c9f52124b0b44fb08181...,directed
4,5,2018-09-26,0x23c848671adf998025cd1d2247fea93f922b6d31,CONGO,Male,Food/Water,0x3cc06c32c88aba379c4bbd4c9ebda585d7574bbf,LINDI,Male,Other,GEP,2.0,0x2141aa9cd1fb6679f0dc2cfee3bd4c69137e96b50809...,directed


In [23]:
all_transactions.to_csv('all_transactions.csv',
                       index=False)